In [2]:
from sqlite3 import DatabaseError, IntegrityError
from typing import List

from sqlalchemy import update, insert
from sqlalchemy.orm import Session

from data_vortex.rightmove_models import RightmoveRentalListing
from data_vortex.database.models import RentalListing

In [3]:
def create_listing(db: Session, rental_listing: RightmoveRentalListing):
    listing_dict = rental_listing.to_orm_dict()
    try:
        existing_listing = (
            db.query(RentalListing)
            .filter(RentalListing.property_id == rental_listing.property_id)
            .first()
        )
        if existing_listing:
            # return RightmoveRentalListing(**existing_listing.dict())
            pass
        else:
            db.execute(
                insert(RentalListing),
                listing_dict,
            )
    except IntegrityError as e:
        db.rollback()
        raise ValueError(f"Integrity error: {e!s}")
    except DatabaseError as e:
        db.rollback()
        raise Exception(f"Database error: {e!s}")

In [4]:
from data_vortex.database.database import SessionLocal, create_database

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
        
db = next(get_db())  # Get a session

In [5]:
existing_listing = (
    db.query(RentalListing)
    .first()
)

OperationalError: (sqlite3.OperationalError) no such table: rental_listings
[SQL: SELECT rental_listings.property_id AS rental_listings_property_id, rental_listings.image_url AS rental_listings_image_url, rental_listings.description AS rental_listings_description, rental_listings.price_amount AS rental_listings_price_amount, rental_listings.price_per AS rental_listings_price_per, rental_listings.price_currency AS rental_listings_price_currency, rental_listings.added_date AS rental_listings_added_date, rental_listings.address AS rental_listings_address, rental_listings.postcode AS rental_listings_postcode, rental_listings.created_date AS rental_listings_created_date 
FROM rental_listings
 LIMIT ? OFFSET ?]
[parameters: (1, 0)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [6]:
from data_vortex.utils.config import settings
settings.DATABASE_URL

'sqlite:///vortex.db'